In [ ]:
!pip install transformers[torch]
# !pip install --upgrade --force-reinstall transformers

In [ ]:
!pip install datasets


In [ ]:
!pip install -U accelerate

In [ ]:
!pip install --upgrade pyarrow

In [ ]:
import re
import pandas as pd
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments


def parse_whatsapp_chat(chat_file):
    with open(chat_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    messages = []
    for line in lines:
        match = re.match(r'(\d{1,2}/\d{1,2}/\d{2}), (\d{1,2}:\d{2}(\s?[APMapm]{2})?) - (.*?): (.*)', line)
        if match:
            date, time, period, sender, message = match.groups()
            messages.append({
                'date': date,
                'time': time,
                'sender': sender,
                'message': message
            })
    return messages

# Load and parse the chat file
chat_file = '/content/WhatsApp Chat with Dilluu.txt'
messages = parse_whatsapp_chat(chat_file)

In [ ]:
friend_messages = [msg['message'] for msg in messages if msg['sender'] == 'Dilluu']

df = pd.DataFrame(friend_messages, columns=['text'])
dataset = Dataset.from_pandas(df)


In [ ]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Add labels to the input data
def tokenize_function(examples):
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_output = tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128)
    # Use input_ids as labels (assuming you want the model to predict the next token in the sequence)
    tokenized_output['labels'] = tokenized_output['input_ids']
    return tokenized_output

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# ... (rest of your code)

Map:   0%|          | 0/5924 [00:00<?, ? examples/s]

In [ ]:
!pip install accelerate -U

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,
    logging_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

Step,Training Loss
500,0.407800
1000,0.190900
1500,0.184100
2000,0.156600
2500,0.148800
3000,0.151700
3500,0.137100


Step,Training Loss
500,0.407800
1000,0.190900
1500,0.184100
2000,0.156600
2500,0.148800
3000,0.151700
3500,0.137100
4000,0.131900


TrainOutput(global_step=4443, training_loss=0.18312750779211373, metrics={'train_runtime': 526.0517, 'train_samples_per_second': 33.784, 'train_steps_per_second': 8.446, 'total_flos': 1160920498176000.0, 'train_loss': 0.18312750779211373, 'epoch': 3.0})

In [ ]:
import torch

In [ ]:
def generate_response(prompt, model, tokenizer, max_length=150, temperature=0.2, top_k=50, top_p=0.9):
    inputs = tokenizer.encode(prompt, return_tensors='pt', add_special_tokens=True).to('cuda')
    # Pass attention mask to the model
    attention_mask = torch.ones(inputs.shape, dtype=torch.long).to('cuda')
    outputs = model.generate(inputs, attention_mask=attention_mask, max_length=max_length, temperature = temperature, top_k = top_k, top_p = top_p, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # response = response[len(prompt):].strip()
    return response

# Example prompt
prompt = "hiii sweetu "
response = generate_response(prompt, model, tokenizer)
print(response)


hiii sweetu 


In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import re
import time

# Function to parse WhatsApp chat data
def parse_whatsapp_chat(chat_file):
    with open(chat_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    messages = []
    for line in lines:
        match = re.match(r'(\d{1,2}/\d{1,2}/\d{2}), (\d{1,2}:\d{2}\s?[APMapm]{2}) - (.*?): (.*)', line)
        if match:
            date, time, sender, message = match.groups()
            messages.append({
                'date': date,
                'time': time,
                'sender': sender,
                'message': message
            })
    return messages

# Load and parse chat file
chat_file = '/content/WhatsApp Chat with Dilluu.txt'
messages = parse_whatsapp_chat(chat_file)

# Filter friend's messages and create a dataset
friend_name = 'Dilluu'  # Replace with your friend's name as it appears in the chat
friend_messages = [msg['message'] for msg in messages if msg['sender'] == friend_name]
df = pd.DataFrame(friend_messages, columns=['text'])
dataset = Dataset.from_pandas(df)

# Load pre-trained model and tokenizer
model_name = "EleutherAI/gpt-neo-125M"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)

# Set pad_token_id to eos_token_id
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
    inputs['labels'] = inputs['input_ids'].clone()
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,  # Enable mixed precision training
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=500,
)

# Create Trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Start training and measure time
start_time = time.time()
trainer.train()
end_time = time.time()

training_time = end_time - start_time
print(f"Training completed in {training_time / 60:.2f} minutes")

# Function to generate responses with style
def generate_styled_response(prompt, model, tokenizer, max_length=150, temperature=0.7, top_k=50, top_p=0.9):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(
        inputs,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Truncate the prompt from the response
    response = response[len(prompt):].strip()
    return response

# Example prompt
prompt = "Hey, how's it going?"

# Define generation parameters
max_length = 150
temperature = 0.7
top_k = 50
top_p = 0.9

response = generate_styled_response(prompt, model, tokenizer, max_length=max_length, temperature=temperature, top_k=top_k, top_p=top_p)
print(response)


Map:   0%|          | 0/5924 [00:00<?, ? examples/s]

Step,Training Loss
500,0.375400
1000,0.186700
1500,0.178000
2000,0.142500
2500,0.135500
3000,0.136900
3500,0.112900
4000,0.107200


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Training completed in 9.39 minutes


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)